# Storing and Accessing objects via Descartes Labs Storage

The `Descartes Labs Catalog API` also supports arbitrary formats of data that don't fall neatly into either a "Raster" or "Vector" data model.

This guide covers the basic methods of the `Blob`, for a more in depth overview of capabilities please see the [Catalog Guide in the Documentation](https://docs.descarteslabs.com/guides/catalog.html).

In [ ]:
import json, os

import descarteslabs as dl
from descarteslabs.catalog import Blob, properties as p

## Creating a `Blob`
The only required attribute for a `Blob` is a unique ID. As with `Products`, `Blobs` also contain a `Namespace`, defaulting to the user's `Organization`. 

Similar to `02 Creating and Managing Products.ipynb`, we will first format a unique `Namespace` with the current user's `default Namespace` ID:

In [ ]:
org = dl.auth.Auth().payload["org"]
user_namespace = dl.auth.Auth().namespace

### _Note on Namespaces_
You can pass any arbitrary string as a specified `Namespace`. Any `Namespace` provided by the user will automatically be prefixed by the user’s `Organization`  and a colon if not already present.

## Storing Arbitrary Data as `Blobs`

`Blobs`  also contain various attributes to search and filter by, such as a `geometry` and `tags`. 

In this first example let's define a `JSON Dictionary` of random data and a geometry to go with it:

In [ ]:
# JSON of random field info
crop_info = {
    "crop": "hops",
    "acreage": 450,
}
# Geometry for the field
field_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-120.4023, 46.551],
            [-120.3859, 46.551],
            [-120.3859, 46.5534],
            [-120.4023, 46.5534],
        ]
    ],
}

Now that we have the data we want to store, we can create create a new `Blob` and call `upload_data()` to upload our dataset. Finally, we call `Blob.save()`:

In [ ]:
blob = Blob(
    name="hop_field_info_demo",
    namespace=f"{org}:{user_namespace}",
    geometry=field_geom,
    tags=["storage_demo"],
)
blob.upload_data(json.dumps(crop_info))
blob.save()
blob

## Storing Files as `Blobs`
If your object is located on disk, you can instead call `Blob.upload('filepath.ext')`

In [ ]:
# Get your organization ID
file_blob = Blob(
    name="yakima_valley",
    namespace=f"{org}:{user_namespace}",
    tags=["storage_demo"],
)
file_blob.upload("data/yakima.geojson")
file_blob.save()
file_blob

## Blob Attributes and Storage Types
The resulting saved `Blob` has several attributes, a few of which are printed below. Note that the `id` is the concatenation of the `Storage Type`, the `Namespace`, and the `Name`. *(Only) The name may contain internal '/' characters.*

If we inspect the `ID` field we can break the namespace down further:
* `data/` is the storage type
* `your-org-name:your-user-hash/` is the `Namespace`
* `hop_field_demo` is the `Blob` object's name

For more information please visit the Docs page on [Storage Types](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.storage_type).

In [ ]:
print("Blob ID:", blob.id)
print("Blob size:", blob.size_bytes)
print("Blob geometry: ", blob.geometry)
print("Blob assigned tags: ", blob.tags)

## Searching `Blobs`
Catalog search methods can be performed across your storage objects, including geospatial searches as well. 

Below are examples of:
* Point intersection
* Polygon intersection
* Tag filter
* Filtering by `Storage Type`

In [ ]:
# Geospatial searches by intersection
## Intersect particular coordinate
print("Point Intersection:")
print(
    [
        b.id
        for b in Blob.search().intersects(
            {"type": "Point", "coordinates": [-120.40, 46.552]}
        )
    ]
)
print("Polygon Intersectsion:")
## Intersecting geometry object
print([b.id for b in Blob.search().intersects(field_geom)])
print("Tag Filter:")
# Filter by tags
print([b.id for b in Blob.search().filter(p.tags == "storage_demo")])
print("Storage Type Filter:")
# Filter by Storage Type
print([b.id for b in Blob.search().filter(p.storage_type == "data")])

We can also use a prefix filter to pick out these new blobs:

In [ ]:
for b in Blob.search().filter(p.name.prefix("foo/")):
    print(b.id)

### Retrieving data from `Blobs`
The `Blob` data may be retrieved, either by downloading directly to a local file or some other file-like object (e.g. an `io.IOBase` object), or directly into memory. Here's a simple download to a file:

In [ ]:
file_blob.download("data/yakima_valley.geojson")

You can also download in raw bytes:

In [ ]:
print(file_blob.data())

### Sharing `Blobs`
As with `Catalog Products`, you can add specific organizations or users as readers to your `Blob` objects to give others access. Simply update the `.readers` list, then save the `Blob` using `.save()`:

In [ ]:
# Adding coworker and org as readers for previously stored GeoTiff file
file_blob.readers = ["email:john.daily@gmail.com", "org:pga-tour"]
file_blob.save()
file_blob.readers

`Blobs` can also transfer ownership, however you cannot use the `email:` variant as an owner. Instead use the new owner's `default Namespace`:

    file_blob.owners.append(f"user:{user_namespace}")
    file_blob.save()

### Deleting `Blobs`
`Blobs` can be deleted using the Catalog module search methods we used earlier.

In [ ]:
# Delete by tags
for b in Blob.search().filter(p.tags == "storage_demo"):
    print(f"Deleting {b.id}")
    b.delete()
os.remove("data/yakima_valley.geojson")